In [6]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)


In [7]:
# Leitura do dataset refinado
diretorio_acidentes = 'data/refined/acidentes_recife.csv'
df_acidentes = pd.read_csv(diretorio_acidentes, sep = ',')
       

C:\Users\Otavio\AppData\Local\Temp\ipykernel_13748\1518368950.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_acidentes = pd.read_csv(diretorio_acidentes, sep = ',')


In [11]:
df_acidentes.head(3)

,Unnamed: 0,data,hora,natureza_acidente,situacao,bairro,endereco,complemento,bairro_cruzamento,tipo,descricao,auto,moto,ciclom,ciclista,pedestre,onibus,caminhao,viatura,outros,vitimas,vitimasfatais,Protocolo,natureza,dia_da_semana,mes,ano
0,0,2020-01-01,NaN,Sem vítimas,CANCELADA,ÁGUA FRIA,RUA SAO SEBASTIAO,EM FRENTE A IGREJA UNIVERSAL,ÁGUA FRIA,COLISÃO,COLISÃO ENTRE DOIS AUTOS S/V,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,Wednesday,1.0,2020.0
1,1,2020-01-01,NaN,Sem vítimas,FINALIZADA,CORDEIRO,AV CAXANGA,NaN,CORDEIRO,ABALROAMENTO LONGITUDINAL,COLISÃO ENTRE DOIS AUTOS S/V,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,Wednesday,1.0,2020.0
2,2,2020-01-01,NaN,Sem vítimas,FINALIZADA,AREIAS,AV RECIFE,LADO OPOSTO AO RESIDENCIAL IGNEZ ANDREAZZA,AREIAS,CHOQUE VEÍCULO PARADO,COLISÃO ENTRE DOIS AUTOS S/V,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,Wednesday,1.0,2020.0
3,3,2020-01-01,NaN,Sem vítimas,FINALIZADA,BOA VIAGEM,AV BOA VIAGEM,EM FRENTE AO 1º JARDIM B. VIAGEM,BOA VIAGEM,COLISÃO TRASEIRA,COLISÃO ENTRE DOIS AUTOS S/V,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,Wednesday,1.0,2020.0
4,4,2020-01-01,NaN,Sem vítimas,FINALIZADA,PAISSANDU,AV GOVERNADOR AGAMENON MAGALHAES,CAIXA ECONÔMICA FEDERAL ILHA DO LEITE,PAISSANDU,ABALROAMENTO TRANSVERSAL,COLISÃOENTRE DOIS AUTOS S/V,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,Wednesday,1.0,2020.0
5,5,2020-01-01,NaN,Sem vítimas,FINALIZADA,BARRO,AV DR JOSE RUFINO,NaN,BARRO,COLISÃO TRASEIRA,COLISÃO ENTRE DOIS AUTOS S/V,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,Wednesday,1.0,2020.0
6,6,2020-01-01,NaN,Sem vítimas,FINALIZADA,PINA,AV CONSELHEIRO AGUIAR,NaN,PINA,CHOQUE OBJETO FIXO,AUTO CHOCOU-SE EM MURO DE UMA RESIDENCIA E EM ...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,Wednesday,1.0,2020.0
7,7,2020-01-01,NaN,Não fatal,FINALIZADA,PINA,AVENIDA CELSO FURTADO (VIA MANGUE),PROX: SHOPPING RIO MAR . ULTIMA CURVA ANTES DO...,PINA,CHOQUE OBJETO FIXO,COLISAO COM DUAS VITIMA. MOTO COM GLADIO DA VIA,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,Wednesday,1.0,2020.0
8,8,2020-01-01,NaN,Não fatal,FINALIZADA,MADALENA,EST DOS REMEDIOS,LADO OPOSTO AO LESSA DE ANDRADE,MADALENA,CHOQUE OBJETO FIXO,PEGEOUT SE CHOCOU COM POSTE E COM O GRADIL DA ...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,Wednesday,1.0,2020.0
9,9,2020-01-01,NaN,Sem vítimas,FINALIZADA,NOVA DESCOBERTA,RUA NOVA DESCOBERTA,NaN,NOVA DESCOBERTA,CHOQUE VEÍCULO PARADO,COLISÃO SEM VÍTIMA,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,Wednesday,1.0,2020.0


# Estatísticas descritivas

# Teste de hipóteses